In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, precision_recall_curve, auc
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
seed_everything()

# Define device (use MPS if available, otherwise use CPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


In [2]:
data_path = 'davis.txt'  # Update with correct path
columns = ['drug_id', 'prot_id', 'SMILES', 'sequence_aa', 'sequence_aa_affinity']
df = pd.read_csv(data_path, sep=' ', names=columns)
# print(df)
# Encode 20 different amino acids with unique numbers
def encode_amino_acids(sequence):
    unique_aa = sorted(set(''.join(sequence)))
    aa_to_int = {aa: i+1 for i, aa in enumerate(unique_aa)}  # Map each AA to a unique number
    return [[aa_to_int[aa] for aa in seq] for seq in sequence]

df['encoded_sequence'] = encode_amino_acids(df['sequence_aa'])
print(df)

        drug_id       prot_id  \
0      11314340          AAK1   
1      11314340   ABL1(E255K)   
2      11314340   ABL1(F317I)   
3      11314340  ABL1(F317I)p   
4      11314340   ABL1(F317L)   
...         ...           ...   
30051    151194           YES   
30052    151194          YSK1   
30053    151194          YSK4   
30054    151194           ZAK   
30055    151194         ZAP70   

                                                  SMILES  \
0      CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...   
1      CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...   
2      CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...   
3      CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...   
4      CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...   
...                                                  ...   
30051  C1=CC=C2C(=C1)C(=NN=C2NC3=CC=C(C=C3)Cl)CC4=CC=...   
30052  C1=CC=C2C(=C1)C(=NN=C2NC3=CC=C(C=C3)Cl)CC4=CC=...   
30053  C1=CC=C2C(=C1)C(=NN=C2NC3=CC=C(C=C3)Cl)CC4=CC=...   
300

In [3]:
from rdkit.Chem import MolFromSmiles
from rdkit.Chem import AllChem

def encode_smiles(smiles_list, n_bits=2):
    fingerprints = []
    for smi in smiles_list:
        mol = MolFromSmiles(smi)
        if mol:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
            fingerprints.append(np.array(fp))
        else:
            fingerprints.append(np.zeros(n_bits))  # Default vector for invalid SMILES
    return np.array(fingerprints)

df['encoded_SMILES'] = list(encode_smiles(df['SMILES']))

In [4]:
import torch
import numpy as np

def process_data(df):
    # 1. Pad the amino acid sequences
    def pad_sequences(sequences, max_len):
        return [
            seq + [0] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len]
            for seq in sequences
        ]
    
    # Get the max length of amino acid sequences for padding
    max_aa_len = max(df['encoded_sequence'].apply(len))

    # Pad the amino acid sequences
    df['padded_sequence'] = pad_sequences(df['encoded_sequence'].tolist(), max_aa_len)

    # Convert padded amino acid sequences to a PyTorch tensor
    X_amino = torch.tensor(df['padded_sequence'].values.tolist(), dtype=torch.float32)

    # 2. Preprocess and pad SMILES sequences
    # Ensure no NaNs or infinities in encoded SMILES
    df['encoded_SMILES'] = df['encoded_SMILES'].apply(lambda x: np.nan_to_num(x))

    # Get the max length of SMILES sequences for padding
    max_smiles_len = max([len(item) for item in df['encoded_SMILES']])

    # Pad the SMILES sequences
    padded_smiles = [np.pad(item, (0, max_smiles_len - len(item))) for item in df['encoded_SMILES']]

    # Convert the padded SMILES sequences to a NumPy array
    X_smiles_np = np.vstack(padded_smiles)

    # 3. Process SMILES in smaller batches to prevent memory overload
    batch_size = 1000  # Adjust the batch size depending on available memory
    chunks = []

    # Split the SMILES data into chunks and convert each chunk to a tensor
    for i in range(0, X_smiles_np.shape[0], batch_size):
        chunk = X_smiles_np[i:i+batch_size]
        chunks.append(torch.tensor(chunk, dtype=torch.float32))  # Convert to float32

    # Concatenate all chunks into one large tensor
    X_smiles = torch.cat(chunks, dim=0)

    return X_amino, X_smiles


## Now 2 cases of splitting, for first, test contains no new proteins, for 2nd one test contains new proteins

## same for drugs, so 4 cases, 4 snippets for train test split,

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming df is the loaded dataset
# Assuming labels are in a column named 'labels'

# Case 1: Protein-based split (70/30 for each protein)
def protein_split_70_30(df):
    train_indices = []
    test_indices = []

    for protein in df['prot_id'].unique():
        protein_indices = df[df['prot_id'] == protein].index
        train_idx, test_idx = train_test_split(protein_indices, test_size=0.3, random_state=42)
        train_indices.extend(train_idx)
        test_indices.extend(test_idx)

    train_df = df.loc[train_indices]
    test_df = df.loc[test_indices]
    train_labels = train_df['sequence_aa_affinity']
    test_labels = test_df['sequence_aa_affinity']
    train_df = train_df.drop(columns=['sequence_aa_affinity'])
    test_df = test_df.drop(columns=['sequence_aa_affinity'])
    return train_df, test_df, train_labels, test_labels

# Case 2: Protein-based split (90/10 with new proteins in test)
def protein_split_new_proteins(df):
    proteins = df['prot_id'].unique()
    train_proteins, test_proteins = train_test_split(proteins, test_size=0.1, random_state=42)

    train_df = df[df['prot_id'].isin(train_proteins)]
    test_df = df[df['prot_id'].isin(test_proteins)]
    train_labels = train_df['sequence_aa_affinity']
    test_labels = test_df['sequence_aa_affinity']
    train_df = train_df.drop(columns=['sequence_aa_affinity'])
    test_df = test_df.drop(columns=['sequence_aa_affinity'])
    return train_df, test_df, train_labels, test_labels

# Case 3: Drug-based split (70/30 for each drug)
def drug_split_70_30(df):
    train_indices = []
    test_indices = []

    for drug in df['drug_id'].unique():
        drug_indices = df[df['drug_id'] == drug].index
        train_idx, test_idx = train_test_split(drug_indices, test_size=0.3, random_state=42)
        train_indices.extend(train_idx)
        test_indices.extend(test_idx)

    train_df = df.loc[train_indices]
    test_df = df.loc[test_indices]
    train_labels = train_df['sequence_aa_affinity']
    test_labels = test_df['sequence_aa_affinity']
    train_df = train_df.drop(columns=['sequence_aa_affinity'])
    test_df = test_df.drop(columns=['sequence_aa_affinity'])
    return train_df, test_df, train_labels, test_labels

# Case 4: Drug-based split (90/10 with new drugs in test)
def drug_split_new_drugs(df):
    drugs = df['drug_id'].unique()
    train_drugs, test_drugs = train_test_split(drugs, test_size=0.1, random_state=42)

    train_df = df[df['drug_id'].isin(train_drugs)]
    test_df = df[df['drug_id'].isin(test_drugs)]
    train_labels = train_df['sequence_aa_affinity']
    test_labels = test_df['sequence_aa_affinity']
    train_df = train_df.drop(columns=['sequence_aa_affinity'])
    test_df = test_df.drop(columns=['sequence_aa_affinity'])
    return train_df, test_df, train_labels, test_labels

# Perform splits
train_df_1, test_df_1, train_labels_1, test_labels_1 = protein_split_70_30(df)
train_df_2, test_df_2, train_labels_2, test_labels_2 = protein_split_new_proteins(df)
train_df_3, test_df_3, train_labels_3, test_labels_3 = drug_split_70_30(df)
train_df_4, test_df_4, train_labels_4, test_labels_4 = drug_split_new_drugs(df)

# Display sizes
print("Protein Split 70/30: Train:", len(train_df_1), "Test:", len(test_df_1))
print("Protein Split New Proteins: Train:", len(train_df_2), "Test:", len(test_df_2))
print("Drug Split 70/30: Train:", len(train_df_3), "Test:", len(test_df_3))
print("Drug Split New Drugs: Train:", len(train_df_4), "Test:", len(test_df_4))


Protein Split 70/30: Train: 20774 Test: 9282
Protein Split New Proteins: Train: 26996 Test: 3060
Drug Split 70/30: Train: 21012 Test: 9044
Drug Split New Drugs: Train: 26962 Test: 3094


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class FeedForwardNN(nn.Module):
    def __init__(self, smiles_dim, amino_dim, hidden_dim):
        super(FeedForwardNN, self).__init__()

        # SMILES branch
        self.fc_smiles = nn.Sequential(
            nn.Linear(smiles_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )

        # Amino acid branch
        self.fc_amino = nn.Sequential(
            nn.Linear(amino_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )

        # Combined branch
        self.fc_combined = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 1),
        )

    def forward(self, smiles, amino):
        # Pass through SMILES branch
        x_smiles = self.fc_smiles(smiles)

        # Pass through Amino acid branch
        x_amino = self.fc_amino(amino)

        # Combine and pass through combined branch
        x_combined = torch.cat((x_smiles, x_amino), dim=1)
        output = self.fc_combined(x_combined)
        return output

# Input dimensions
X_amino_1, X_smiles_1 = process_data(train_df_1)
X_amino_2, X_smiles_2 = process_data(train_df_2)
X_amino_3, X_smiles_3 = process_data(train_df_3)
X_amino_4, X_smiles_4 = process_data(train_df_4)

# Hidden dimension can be adjusted as needed
hidden_dim = 128

# Define models for each split
model_case1 = FeedForwardNN(X_smiles_1.shape[1], X_amino_1.shape[1], hidden_dim).to(device)
model_case2 = FeedForwardNN(X_smiles_2.shape[1], X_amino_2.shape[1], hidden_dim).to(device)
model_case3 = FeedForwardNN(X_smiles_3.shape[1], X_amino_3.shape[1], hidden_dim).to(device)
model_case4 = FeedForwardNN(X_smiles_4.shape[1], X_amino_4.shape[1], hidden_dim).to(device)

# Loss function and optimizer for each model
criterion_case1 = nn.MSELoss()
criterion_case2 = nn.MSELoss()
criterion_case3 = nn.MSELoss()
criterion_case4 = nn.MSELoss()
optimizer_case1 = optim.Adam(model_case1.parameters(), lr=0.001)
optimizer_case2 = optim.Adam(model_case2.parameters(), lr=0.001)
optimizer_case3 = optim.Adam(model_case3.parameters(), lr=0.001)
optimizer_case4 = optim.Adam(model_case4.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, train_df, test_df, train_labels, epochs=400):
    model.train()
    
    # Convert training data to tensors
    X_smiles_train = torch.tensor(train_df['encoded_SMILES'].tolist(), dtype=torch.float32).to(device)
    X_amino_train = torch.tensor(train_df['padded_sequence'].tolist(), dtype=torch.float32).to(device)
    y_train = torch.tensor(train_labels.tolist(), dtype=torch.float32).to(device)

    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(X_smiles_train, X_amino_train).squeeze()
        
        # Calculate loss
        loss = criterion(predictions, y_train)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')


# Train each model with respective splits
print("Training model for case 1...")
train_model(model_case1, criterion_case1, optimizer_case1, train_df_1, test_df_1, train_labels_1)

print("Training model for case 2...")
train_model(model_case2, criterion_case2, optimizer_case2, train_df_2, test_df_2, train_labels_2)

print("Training model for case 3...")
train_model(model_case3, criterion_case3, optimizer_case3, train_df_3, test_df_3, train_labels_3)

print("Training model for case 4...")
train_model(model_case4, criterion_case4, optimizer_case4, train_df_4, test_df_4, train_labels_4)


Training model for case 1...


/var/folders/dy/nh23qmtn6n3g6c50qc9w1fhm0000gn/T/ipykernel_24016/310495481.py:90: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  X_smiles_train = torch.tensor(train_df['encoded_SMILES'].tolist(), dtype=torch.float32).to(device)


Epoch 10/400, Loss: 22.2174
Epoch 20/400, Loss: 16.7940
Epoch 30/400, Loss: 11.9762
Epoch 40/400, Loss: 7.4259
Epoch 50/400, Loss: 3.7405
Epoch 60/400, Loss: 1.6152
Epoch 70/400, Loss: 0.8071
Epoch 80/400, Loss: 0.7271
Epoch 90/400, Loss: 0.7403
Epoch 100/400, Loss: 0.7088
Epoch 110/400, Loss: 0.7072
Epoch 120/400, Loss: 0.7013
Epoch 130/400, Loss: 0.7005
Epoch 140/400, Loss: 0.6999
Epoch 150/400, Loss: 0.6997
Epoch 160/400, Loss: 0.6997
Epoch 170/400, Loss: 0.6996
Epoch 180/400, Loss: 0.6996
Epoch 190/400, Loss: 0.6996
Epoch 200/400, Loss: 0.6996
Epoch 210/400, Loss: 0.6996
Epoch 220/400, Loss: 0.6996
Epoch 230/400, Loss: 0.6996
Epoch 240/400, Loss: 0.6996
Epoch 250/400, Loss: 0.6996
Epoch 260/400, Loss: 0.6996
Epoch 270/400, Loss: 0.6996
Epoch 280/400, Loss: 0.6996
Epoch 290/400, Loss: 0.6996
Epoch 300/400, Loss: 0.6996
Epoch 310/400, Loss: 0.6996
Epoch 320/400, Loss: 0.6996
Epoch 330/400, Loss: 0.6996
Epoch 340/400, Loss: 0.6996
Epoch 350/400, Loss: 0.6996
Epoch 360/400, Loss: 0.699

In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np
import torch

# Concordance Index (CI) implementation
def concordance_index(y_true, y_pred):
    n = 0
    h_sum = 0
    for i in range(len(y_true)):
        for j in range(i + 1, len(y_true)):
            if y_true[i] != y_true[j]:
                n += 1
                if (y_pred[i] > y_pred[j] and y_true[i] > y_true[j]) or \
                   (y_pred[i] < y_pred[j] and y_true[i] < y_true[j]):
                    h_sum += 1
                elif y_pred[i] == y_pred[j]:
                    h_sum += 0.5
    return h_sum / n if n > 0 else 0.0

# Updated Evaluation Function
# Updated Evaluation Function
def evaluate_model(model, X_smiles_test, X_amino_test, y_test, device):
    model.eval()
    with torch.no_grad():
        # Convert y_test (Pandas Series) to NumPy array before passing to torch.tensor
        y_test_numpy = y_test.to_numpy()  # Convert to NumPy array
        y_test_tensor = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)
        
        # Predictions
        predictions = model(X_smiles_test, X_amino_test)
        
        # Move predictions to CPU and convert to NumPy (fixed)
        predictions = predictions.cpu().detach().numpy()  # Move to CPU before numpy conversion
        
        # Convert y_test_tensor to NumPy for comparison
        y_test_np = y_test_tensor.cpu().detach().numpy()  # Ensure y_test_tensor is on CPU
        
        # Metrics
#         ci = concordance_index(y_test_np, predictions)
        mse = mean_squared_error(y_test_np, predictions)
        r2 = r2_score(y_test_np, predictions)
#         pearson_corr, _ = pearsonr(y_test_np, predictions)
        
        # Print Metrics
#         print(f'Concordance Index (CI): {ci:.4f}')
        print(f'Mean Squared Error (MSE): {mse:.4f}')
        print(f'R^2: {r2:.4f}')
#         print(f'Pearson Correlation Coefficient (R): {pearson_corr:.4f}')

        
X_amino_t_1, X_smiles_t_1 = process_data(test_df_1)
X_amino_t_2, X_smiles_t_2 = process_data(test_df_2)
X_amino_t_3, X_smiles_t_3 = process_data(test_df_3)
X_amino_t_4, X_smiles_t_4 = process_data(test_df_4)

X_amino_t_1 = X_amino_t_1.to(device)
X_amino_t_2 = X_amino_t_2.to(device)
X_amino_t_3 = X_amino_t_3.to(device)
X_amino_t_4 = X_amino_t_4.to(device)

X_smiles_t_1 = X_smiles_t_1.to(device)
X_smiles_t_2 = X_smiles_t_2.to(device)
X_smiles_t_3 = X_smiles_t_3.to(device)
X_smiles_t_4 = X_smiles_t_4.to(device)



# X_smiles_t_1 = torch.tensor(test_df_1['encoded_SMILES'].tolist(), dtype=torch.float32).to(device)
# X_amino_t_1 = torch.tensor(test_df_1['padded_sequence'].tolist(), dtype=torch.float32).to(device)
# X_smiles_t_2 = torch.tensor(test_df_2['encoded_SMILES'].tolist(), dtype=torch.float32).to(device)
# X_amino_t_2 = torch.tensor(test_df_2['padded_sequence'].tolist(), dtype=torch.float32).to(device)
# X_smiles_t_3 = torch.tensor(test_df_3['encoded_SMILES'].tolist(), dtype=torch.float32).to(device)
# X_amino_t_3 = torch.tensor(test_df_3['padded_sequence'].tolist(), dtype=torch.float32).to(device)
# X_smiles_t_4 = torch.tensor(test_df_4['encoded_SMILES'].tolist(), dtype=torch.float32).to(device)
# X_amino_t_4 = torch.tensor(test_df_4['padded_sequence'].tolist(), dtype=torch.float32).to(device)

# Evaluate Models
print("Evaluating Model for Case 1...")
evaluate_model(model_case1, X_smiles_t_1, X_amino_t_1, test_labels_1, device)

print("\nEvaluating Model for Case 2...")
evaluate_model(model_case2, X_smiles_t_2, X_amino_t_2, test_labels_2, device)

print("\nEvaluating Model for Case 3...")
evaluate_model(model_case3, X_smiles_t_3, X_amino_t_3, test_labels_3, device)

print("\nEvaluating Model for Case 4...")
evaluate_model(model_case4, X_smiles_t_4, X_amino_t_4, test_labels_4, device)


Evaluating Model for Case 1...
Mean Squared Error (MSE): 2015.7471
R^2: -2625.0547

Evaluating Model for Case 2...


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (3060x2027 and 2549x128)